In [12]:
pip install PyPDF2 pdfplumber transformers torch ollama requests

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import json
import csv
import PyPDF2
import pdfplumber
import requests
from typing import List, Dict, Any, Optional
from transformers import pipeline
import ollama


In [19]:
class MCQGenerator:
    def __init__(self, model_type: str = "openai", api_key: str = None, model_name: str = "gpt-4"):
        """
        Initialize the MCQ Generator with specified model.
        
        Args:
            model_type: Type of model to use ('openai', 'ollama', 'huggingface')
            api_key: API key for the service
            model_name: Specific model name
        """
        self.model_type = model_type
        self.model_name = model_name
        
        if model_type == "openai" and api_key:
            self.client = OpenAI(api_key=api_key)
        elif model_type == "huggingface":
            self.generator = pipeline('text-generation', model=model_name)
        elif model_type == "ollama":
            # Ensure the model is available in Ollama
            try:
                ollama.list()
            except:
                print("Warning: Ollama might not be running")

In [5]:
def extract_text_from_pdf(self, pdf_path: str, use_pdfplumber: bool = True) -> str:
        """
        Extract text from PDF file.
        
        Args:
            pdf_path: Path to the PDF file
            use_pdfplumber: Whether to use pdfplumber (better for complex layouts)
            
        Returns:
            Extracted text as string
        """
        text = ""
        
        try:
            if use_pdfplumber:
                with pdfplumber.open(pdf_path) as pdf:
                    for page in pdf.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
            else:
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page in pdf_reader.pages:
                        text += page.extract_text() + "\n"
                        
        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            
        return text

In [6]:
def chunk_text(self, text: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> List[str]:
        """
        Split text into overlapping chunks for processing.
        
        Args:
            text: Input text to chunk
            chunk_size: Size of each chunk in characters
            chunk_overlap: Overlap between chunks
            
        Returns:
            List of text chunks
        """
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append(chunk)
            start += chunk_size - chunk_overlap
            
        return chunks

In [7]:
def retrieve_relevant_chunks(self, chunks: List[str], topic: str, top_k: int = 3) -> List[str]:
        """
        Simple retrieval function to find relevant chunks based on topic.
        In production, you might use vector databases like FAISS or ChromaDB.
        
        Args:
            chunks: List of text chunks
            topic: Topic to search for
            top_k: Number of top chunks to return
            
        Returns:
            List of relevant chunks
        """
        # Simple keyword-based retrieval
        topic_lower = topic.lower()
        scored_chunks = []
        
        for chunk in chunks:
            score = chunk.lower().count(topic_lower)
            scored_chunks.append((score, chunk))
        
        # Sort by relevance score and return top_k
        scored_chunks.sort(reverse=True)
        return [chunk for _, chunk in scored_chunks[:top_k]]

In [8]:
def generate_mcq_with_llm(self, context: str, topic: str, num_questions: int = 1) -> List[Dict[str, Any]]:
        """
        Generate MCQs using the selected LLM.
        
        Args:
            context: Context text for question generation
            topic: Specific topic for questions
            num_questions: Number of questions to generate
            
        Returns:
            List of MCQ dictionaries
        """
        prompt = f"""
        Based on the following context, generate {num_questions} multiple-choice question(s) about {topic}.
        
        Context:
        {context}
        
        For each question, provide:
        1. A clear question
        2. Four options (A, B, C, D)
        3. The correct answer (just the letter)
        4. Brief explanatory feedback
        
        Format the response as a JSON array where each object has:
        - "question": the question text
        - "options": a dictionary with keys "A", "B", "C", "D"
        - "correct_answer": the correct option letter
        - "feedback": explanatory text
        
        Generate only the JSON array, no additional text.
        """
        
        if self.model_type == "openai":
            return self._generate_with_openai(prompt)
        elif self.model_type == "ollama":
            return self._generate_with_ollama(prompt)
        elif self.model_type == "huggingface":
            return self._generate_with_huggingface(prompt)
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

In [9]:
def _generate_with_openai(self, prompt: str) -> List[Dict[str, Any]]:
        """Generate MCQs using OpenAI API."""
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are an expert educational content creator. Generate accurate, clear multiple-choice questions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2000
            )
            
            content = response.choices[0].message.content.strip()
            return json.loads(content)
            
        except Exception as e:
            print(f"Error generating with OpenAI: {e}")
            return []

In [10]:
def _generate_with_ollama(self, prompt: str) -> List[Dict[str, Any]]:
        """Generate MCQs using Ollama."""
        try:
            response = ollama.generate(
                model=self.model_name,
                prompt=prompt,
                options={'temperature': 0.7}
            )
            
            content = response['response'].strip()
            return json.loads(content)
            
        except Exception as e:
            print(f"Error generating with Ollama: {e}")
            return []

In [11]:
 def _generate_with_huggingface(self, prompt: str) -> List[Dict[str, Any]]:
        """Generate MCQs using Hugging Face transformers."""
        try:
            response = self.generator(
                prompt,
                max_length=2000,
                temperature=0.7,
                do_sample=True,
                num_return_sequences=1
            )

SyntaxError: incomplete input (4266373682.py, line 10)

In [ ]:
content = response[0]['generated_text'].replace(prompt, '').strip()
            # Extract JSON from response
            start_idx = content.find('[')
            end_idx = content.rfind(']') + 1
            if start_idx != -1 and end_idx != 0:
                json_str = content[start_idx:end_idx]
                return json.loads(json_str)
            return []
            
        except Exception as e:
            print(f"Error generating with Hugging Face: {e}")
            return []

In [ ]:
def save_to_json(self, mcqs: List[Dict[str, Any]], filename: str):
        """Save MCQs to JSON file."""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(mcqs, f, indent=2, ensure_ascii=False)
            print(f"MCQs saved to {filename}")
        except Exception as e:
            print(f"Error saving to JSON: {e}")
    

In [ ]:
def save_to_csv(self, mcqs: List[Dict[str, Any]], filename: str):
        """Save MCQs to CSV file."""
        try:
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                # Write header
                writer.writerow(['question', 'option_A', 'option_B', 'option_C', 'option_D', 'correct_answer', 'feedback'])
                
                for mcq in mcqs:
                    writer.writerow([
                        mcq['question'],
                        mcq['options']['A'],
                        mcq['options']['B'],
                        mcq['options']['C'],
                        mcq['options']['D'],
                        mcq['correct_answer'],
                        mcq['feedback']
                    ])
            print(f"MCQs saved to {filename}")
        except Exception as e:
            print(f"Error saving to CSV: {e}")

In [ ]:
def generate_moodle_format(self, mcqs: List[Dict[str, Any]]) -> str:
        """
        Convert MCQs to Moodle GIFT format for easy import.
        
        Returns:
            String in GIFT format
        """
        gift_text = ""
        for mcq in mcqs:
            question = mcq['question'].replace('{', '\\{').replace('}', '\\}')
            correct_option = mcq['options'][mcq['correct_answer']]
            
            gift_text += f"::{question}::"
            gift_text += "{" + correct_option.replace('=', '\\=')
            
            # Add other options as wrong answers
            for option, text in mcq['options'].items():
                if option != mcq['correct_answer']:
                    gift_text += f" ~{text.replace('=', '\\=')}"
            
            gift_text += "}\n\n"
        
        return gift_text

In [ ]:
def main():
    """Example usage of the MCQ Generator."""
    
    # Initialize the generator (choose one model type)
    generator = MCQGenerator(
        model_type="openai",  # or "ollama", "huggingface"
        api_key="your-openai-api-key",  # Replace with your API key
        model_name="gpt-4"
    )
    

In [ ]:
# Extract text from PDF
    pdf_path = "sample_document.pdf"  # Replace with your PDF path
    if os.path.exists(pdf_path):
        text = generator.extract_text_from_pdf(pdf_path)
        print(f"Extracted {len(text)} characters from PDF")
        
        # Chunk the text
        chunks = generator.chunk_text(text)
        print(f"Created {len(chunks)} chunks")
        
        # Retrieve relevant chunks for a topic
        topic = "artificial intelligence"
        relevant_chunks = generator.retrieve_relevant_chunks(chunks, topic)
        context = "\n".join(relevant_chunks)
        

In [ ]:
# Generate MCQs
        mcqs = generator.generate_mcq_with_llm(context, topic, num_questions=3)
        
        if mcqs:
            print(f"Generated {len(mcqs)} MCQs")
            
            # Display generated MCQs
            for i, mcq in enumerate(mcqs, 1):
                print(f"\n--- Question {i} ---")
                print(f"Q: {mcq['question']}")
                for option, text in mcq['options'].items():
                    print(f"  {option}: {text}")
                print(f"Correct: {mcq['correct_answer']}")
                print(f"Feedback: {mcq['feedback']}")
            
            # Save to files
            generator.save_to_json(mcqs, "generated_mcqs.json")
            generator.save_to_csv(mcqs, "generated_mcqs.csv")
            
            # Generate Moodle format
            moodle_format = generator.generate_moodle_format(mcqs)
            with open("moodle_questions.gift", "w", encoding="utf-8") as f:
                f.write(moodle_format)
            print("\nMoodle GIFT format saved to moodle_questions.gift")
            
        else:
            print("No MCQs were generated")
    else:
        print("PDF file not found. Using sample context...")
        
        # Fallback: Generate from sample context
        sample_context = """
        Retrieval-Augmented Generation (RAG) is a technique that combines retrieval-based methods 
        with generative AI models. It enhances the accuracy and factual consistency of generated 
        content by first retrieving relevant information from a knowledge source, then using that 
        information to generate responses. This approach is particularly useful for applications 
        requiring up-to-date or domain-specific knowledge, as it grounds the generation in factual data.
        
        Large Language Models (LLMs) like GPT-4 are trained on vast amounts of text data and can 
        generate human-like responses. However, they may sometimes produce inaccurate or outdated 
        information. RAG addresses this limitation by providing the model with current, relevant 
        context from external sources before generation.
        """
        
        mcqs = generator.generate_mcq_with_llm(sample_context, "RAG and LLMs", num_questions=2)
        
        if mcqs:
            for i, mcq in enumerate(mcqs, 1):
                print(f"\n--- Question {i} ---")
                print(f"Q: {mcq['question']}")
                for option, text in mcq['options'].items():
                    print(f"  {option}: {text}")
                print(f"Correct: {mcq['correct_answer']}")
                print(f"Feedback: {mcq['feedback']}")

if __name__ == "__main__":
    main()

In [ ]:
# For OpenAI
generator = MCQGenerator(model_type="openai", api_key="your-key")

# For Ollama (local models)
generator = MCQGenerator(model_type="ollama", model_name="mistral")

# For Hugging Face
generator = MCQGenerator(model_type="huggingface", model_name="microsoft/DialoGPT-medium")